<font size=7 face="Courier"> Latencies Source Code

# Set Up Notebook

import stuff

In [1]:
import numpy as np                     # Packages for Data Analysis
import pandas as pd
import random as rand

import matplotlib.pyplot as plt       # Packages for making plots
from matplotlib.patches import FancyArrow
import matplotlib.colors as mcolors
import matplotlib.patches as patches
from matplotlib.patches import Patch, Circle
from matplotlib.lines import Line2D
from scipy.ndimage import gaussian_filter1d

import braingeneers                  # Braigneneers packages for analysis
from braingeneers.analysis.analysis import SpikeData, read_phy_files, load_spike_data, burst_detection, randomize_raster
import braingeneers.data.datasets_electrophysiology as ephys
import diptest                       # Package to run statistical test                


import ipywidgets as ipw             # Packages for makings GUI's
from ipywidgets import interact, interactive, fixed, interact_manual  # package for interactive widgets from IPython.display import HTML, display, Javascript, clear_output,
from IPython.display import HTML, display, Javascript, clear_output


# Helper Functions

## Mean Firing Rates

In [2]:
def calculate_mean_firing_rates(spike_data):
    mean_firing_rates = []
    for neuron_spikes in spike_data.train:
        num_spikes = len(neuron_spikes)
        time_duration = spike_data.length / 1000  # Assuming spike times are in milliseconds
        firing_rate = num_spikes / time_duration
        mean_firing_rates.append(firing_rate)
    return np.array(mean_firing_rates)

## Plot Latency Histogram

In [3]:
def plotLatencyHistograms(lates):
    
    mean_latency = np.mean(latencies)     # Get mean and SD
    std_latency = np.std(latencies)
    cutoff = 2 * std_latency           # remove outliers
    lates_filtered = [latency for latency in latencies if abs(latency - mean_latency) <= cutoff]
    plots[1,1].hist(lates_filtered, bins=12, alpha=0.7, label='Latency')
    plots[1,1].axvline(mean_latency, color='red', linestyle='dashed', linewidth=2, label='Mean')
    plots[1,1].axvline(mean_latency - std_latency, color='green', linestyle='dashed', linewidth=2, label='Std -')
    plots[1,1].axvline(mean_latency + std_latency, color='green', linestyle='dashed', linewidth=2, label='Std +')
    plots[1,1].axvline(0, color='black', linestyle='dashed', linewidth=0.5, label='Std +')
    axs[i].set_xlim(-1*(abs(mean_latency)+cutoff), abs(mean_latency) + cutoff) 
    plots[1,1].set_xlabel("Latency (ms)")
    plots[1,1].set_ylabel("Count")
    plots[1,1].set_title(f"Latency Histogram - Neuron Pair {neuron1} and {neuron2}")
    plots[1,1].legend()


# plot Waveforms in Layout View

Plots the waveforms of a neuron in layout view

In [ ]:
def plotWaveformLayout( neuron_id, sd, axs=None):
    if axs is None:
        fig, axs = plt.subplots(1, 1, figsize=(8,8) )
    #axs.set_title(title, fontsize=12)

    
    #setting parameters that were previously function variables in Sury's code
    data = sd.neuron_data[0][neuron_id]
    k=neuron_id
    ch=data["channel"]
    position=data["position"]
    temp_chs=list(data["neighbor_channels"])
    temp_pos=data["neighbor_positions"]; templates=data["neighbor_templates"]; filename=f"{id}"; nelec=2
    
    axs.set_title(f"{filename} cluster {k} channel {ch} {position}")
    assert len(temp_chs) == len(temp_pos) == len(templates), "Input length should be the same!"
    nums = len(temp_chs)
    pitch = 17.5
    axs.scatter(position[0], position[1], linewidth=10, alpha=0.2, color='grey')
    axs.text(position[0], position[1], str(position), color="g", fontsize=12)
    # set same scaling to the insets
    index = temp_chs.index(ch)
    ylim_min = min(templates[index])
    ylim_max = max(templates[index])
    # choose channels that are close to the center channel
    for i in range(nums):
        chn_pos = temp_pos[i]
        if position[0] - nelec * pitch <= chn_pos[0] <= position[0] + nelec * pitch \
                and position[1] - nelec * pitch <= chn_pos[1] <= position[1] + nelec * pitch:
            # axs.scatter(chn_pos[0], chn_pos[1], color='w')
            axin = axs.inset_axes([chn_pos[0]-5, chn_pos[1]-5, 15, 20], transform=axs.transData)
            axin.plot(templates[i], color='k', linewidth=2, alpha=0.7)
            axin.set_ylim([ylim_min - ylim_margin, ylim_max + ylim_margin])
            axin.set_axis_off()
    # axs.legend(loc="upper right", fontsize=12)
    # axs.xaxis.set_visible(False)
    # axs.yaxis.set_visible(False)
    axs.set_xlim(position[0]-1.5*nelec*pitch, position[0]+1.5*nelec*pitch)
    axs.set_ylim(position[1]-1.5*nelec*pitch, position[1]+1.5*nelec*pitch)
    axs.invert_yaxis()
    return axs

In [6]:
# def plotWaveformLayout(axs, neuron_id, sd):
#     #setting parameters that were previously function variables in Sury's code
#     data = sd.neuron_data[0][neuron_id]
#     k=neuron_id, ch=data["channel"], position=data["position"], temp_chs=list(data["neighbor_channels"])
#     temp_pos=data["neighbor_positions"]; templates=data["neighbor_templates"]; filename=f"{id}"; nelec=2
    
#     axs.set_title(f"{filename} cluster {k} channel {ch} {position}")
#     assert len(temp_chs) == len(temp_pos) == len(templates), "Input length should be the same!"
#     nums = len(temp_chs)
#     pitch = 17.5
#     axs.scatter(position[0], position[1], linewidth=10, alpha=0.2, color='grey')
#     axs.text(position[0], position[1], str(position), color="g", fontsize=12)
#     # set same scaling to the insets
#     index = temp_chs.index(ch)
#     ylim_min = min(templates[index])
#     ylim_max = max(templates[index])
#     # choose channels that are close to the center channel
#     for i in range(nums):
#         chn_pos = temp_pos[i]
#         if position[0] - nelec * pitch <= chn_pos[0] <= position[0] + nelec * pitch \
#                 and position[1] - nelec * pitch <= chn_pos[1] <= position[1] + nelec * pitch:
#             # axs.scatter(chn_pos[0], chn_pos[1], color='w')
#             axin = axs.inset_axes([chn_pos[0]-5, chn_pos[1]-5, 15, 20], transform=axs.transData)
#             axin.plot(templates[i], color='k', linewidth=2, alpha=0.7)
#             axin.set_ylim([ylim_min - ylim_margin, ylim_max + ylim_margin])
#             axin.set_axis_off()
#     # axs.legend(loc="upper right", fontsize=12)
#     # axs.xaxis.set_visible(False)
#     # axs.yaxis.set_visible(False)
#     axs.set_xlim(position[0]-1.5*nelec*pitch, position[0]+1.5*nelec*pitch)
#     axs.set_ylim(position[1]-1.5*nelec*pitch, position[1]+1.5*nelec*pitch)
#     axs.invert_yaxis()
#     return axs

In [7]:
# fig, axs = plt.subplots()
# plot_inset(axs, neuron_id)

# Latency Layout Plots

## <font color="blue"> Latency Layout Plot

Plot any sum of latencies provided to it. Input is a dictionary with the key is the latency pair and the value are the latencies

In [8]:
def plotLatencyLayout( sd, pairs_lates_directed ):
    
    # Scatter plot of neurons
    plt.figure(figsize=(8, 8))
    neuron_x = []
    neuron_y = []
    for neuron in sd.neuron_data[0].values():
        neuron_x.append(neuron['position'][0])
        neuron_y.append(neuron['position'][1])
    plt.scatter(neuron_x, neuron_y, alpha=0.15, c='grey')

    # Plot Latencies
    for pair, entries in pairs_lates_directed.items():
        start_i = pair[0]
        end_i = pair[1]
        arrow = FancyArrow(
            neuron_x[end_i], neuron_y[end_i],
            neuron_x[start_i] - neuron_x[end_i], neuron_y[start_i] - neuron_y[end_i],
            length_includes_head=True, head_width=25,
            linewidth=1, color="red", alpha=0.7, edgecolor="red", facecolor="red")
        plt.gca().add_patch(arrow)

    plt.xlabel('um')
    plt.ylabel('um')
    plt.title("Directionality plot")
    plt.show()

## <font color="blue">plot Latency Pair Footprint 

In [9]:
def plotLatencyPairFootprint( sd, pairs_lates_directed ):
    
    # Scatter plot of neurons
    plt.figure(figsize=(8, 8))
    neuron_x = []
    neuron_y = []
    for neuron in sd.neuron_data[0].values():
        neuron_x.append(neuron['position'][0])
        neuron_y.append(neuron['position'][1])
    plt.scatter(neuron_x, neuron_y, alpha=0.10, c='grey')

    # Plot Latencies
    for pair, entries in pairs_lates_directed.items():
        start_i = pair[0]
        end_i = pair[1]
        
        # Plot neuron geographic location
        for neighbor in  sd.neuron_data[0][start_i]['neighbor_positions']:
            plt.scatter( [neighbor[0]+rand.random()*13], [neighbor[1]+rand.random()*10], alpha=0.40, c='blue')
        for neighbor in  sd.neuron_data[0][end_i]['neighbor_positions']:
            plt.scatter( [neighbor[0]+rand.random()*13], [neighbor[1]+rand.random()*10], alpha=0.40, c='green')
        
        arrow = FancyArrow(
            neuron_x[end_i], neuron_y[end_i],
            neuron_x[start_i] - neuron_x[end_i], neuron_y[start_i] - neuron_y[end_i],
            length_includes_head=True, head_width=25,
            linewidth=1, color="red", alpha=0.7, edgecolor="red", facecolor="red")
        plt.gca().add_patch(arrow)

    plt.xlabel('um')
    plt.ylabel('um')
    plt.title("Directionality plot")
    plt.show()

# <font color="blue">plot Latency Pair Footprint 2 - General

In [10]:
def plotPairFootprint( start_i, end_i, sd):
    
    # Scatter plot of neurons
    plt.figure(figsize=(8, 8))
    neuron_x = []
    neuron_y = []
    for neuron in sd.neuron_data[0].values():
        neuron_x.append(neuron['position'][0])
        neuron_y.append(neuron['position'][1])
    plt.scatter(neuron_x, neuron_y, alpha=0.10, c='grey')
  
    # Plot neuron geographic location
    for neighbor in  sd.neuron_data[0][start_i]['neighbor_positions']:
        plt.scatter( [neighbor[0]+rand.random()*13], [neighbor[1]+rand.random()*10], alpha=0.40, c='blue')
    for neighbor in  sd.neuron_data[0][end_i]['neighbor_positions']:
        plt.scatter( [neighbor[0]+rand.random()*13], [neighbor[1]+rand.random()*10], alpha=0.40, c='green')

    arrow = FancyArrow(
        neuron_x[end_i], neuron_y[end_i],
        neuron_x[start_i] - neuron_x[end_i], neuron_y[start_i] - neuron_y[end_i],
        length_includes_head=True, head_width=25,
        linewidth=1, color="red", alpha=0.7, edgecolor="red", facecolor="red")
    plt.gca().add_patch(arrow)

    plt.xlabel('um')
    plt.ylabel('um')
    plt.title("Directionality plot")
    plt.show()

## <font color="brown">Old- Grouped Latencies Plot

This is a plot of the latencies, but with colors denoted different groups of arrows

In [11]:
def plotLatencyLayoutGroups(sd, pairs_lates_directed):

    neuron_x = []
    neuron_y = []
    for neuron in sd.neuron_data[0].values():
        neuron_x.append(neuron['position'][0])
        neuron_y.append(neuron['position'][1])

    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib.patches import FancyArrow

    # Assuming you have 'pairs_lates_directed' dictionary and neuron_x, neuron_y arrays
    # pairs_lates_directed = { (pair): np.array(entries), ... }
    # neuron_x = ...  # Your x-coordinates
    # neuron_y = ...  # Your y-coordinates

    direction_threshold = 0.65

    plt.figure(figsize=(8, 8))

    plt.scatter(neuron_x, neuron_y, alpha=0.15, c='grey')

    count=0
    for pair, entries in pairs_lates_directed.items():
        lates_i = np.array(entries)

        if len(lates_i) == 0:
            continue

        percent_neg = np.sum(lates_i < 0) / len(lates_i)
        percent_pos = np.sum(lates_i > 0) / len(lates_i)

        # Check the direction and conditions
        is_pos = np.median(lates_i) > 0
        if is_pos and percent_pos > direction_threshold:
            count +=1
            start_i = pair[0]
            end_i = pair[1]

            arrow_color = "black"
            if neuron_y[start_i] < 500 or neuron_y[end_i] < 500:
                arrow_color = "green"

            if neuron_x[start_i] < neuron_x[end_i] and (neuron_y[start_i] > 500 or neuron_y[end_i] > 500):
                arrow_color = "red"

            if neuron_x[start_i] > neuron_x[end_i] and (neuron_y[start_i] > 500 or neuron_y[end_i] > 500):
                arrow_color = "blue"

            arrow = FancyArrow(
                neuron_x[end_i], neuron_y[end_i],
                neuron_x[start_i] - neuron_x[end_i], neuron_y[start_i] - neuron_y[end_i],
                length_includes_head=True, head_width=25,
                linewidth=1, color=arrow_color, alpha=0.7, edgecolor=arrow_color, facecolor=arrow_color)
            plt.gca().add_patch(arrow)

    plt.xlabel('um')
    plt.ylabel('um')
    plt.title("Directionality plot")
    print(count)
    plt.show()

## <font color="blue">Latency Pair Analysis

Deep analysis of a single pair of latencies

In [12]:
def plotLatencyPairAnalysis(num):
    
    import warnings
    warnings.filterwarnings("ignore")

    # Layout Plot
    pairs_lates_list = list(pairs_lates_clean.items())
    to_graph = {pairs_lates_list[num][0]:pairs_lates_list[num][1]}
    plotLatencyPairFootprint(sd,to_graph)
    
    # Summary Stats
    start_i = pairs_lates_list[num][0][0]
    end_i = pairs_lates_list[num][0][1]
    latencies = pairs_lates_list[num][1]
    
    print( "Number of Latencies", len(latencies) )
    print( "Mean Latency", round(np.mean(latencies), 3) )
    print( "Median Latency", round(np.median(latencies), 3) )
    #print( "Latency Probability", {round(100*(len(lates_filtered)/len(lates_raw)))})
    print("STTC", sttc[start_i][end_i])
    print("Diptest P-val", round( diptest.diptest(latencies)[1] , 3) )
    print( "Latency Probability", round(len(latencies)/len(sd.train[start_i]), 3) )
    print(".")
    print( "Sender Neuron", start_i)
    print( "Sender   Firing Rate", round(firing_rates[start_i], 3) )
    print(".")
    print("Receiver Neuron", end_i)
    print( "Receiver Firing Rate", round(firing_rates[end_i], 3) )
    
    # Plot other graphs
    figs, plots = plt.subplots(nrows=3,ncols=2,figsize=(14,10))

    mean_latency = np.mean(latencies)     # Get mean and SD
    std_latency = np.std(latencies)
    cutoff = 2 * std_latency           # remove outliers
    lates_filtered = [latency for latency in latencies if abs(latency - mean_latency) <= cutoff]
    plots[0,0].hist(lates_filtered, bins=12, alpha=0.7, label='Latency')
    plots[0,0].axvline(mean_latency, color='red', linestyle='dashed', linewidth=2, label='Mean')
    plots[0,0].axvline(mean_latency - std_latency, color='green', linestyle='dashed', linewidth=2, label='Std -')
    plots[0,0].axvline(mean_latency + std_latency, color='green', linestyle='dashed', linewidth=2, label='Std +')
    plots[0,0].axvline(0, color='black', linestyle='dashed', linewidth=0.5, label='Std +')
    plots[0,0].set_xlim(-1*(abs(mean_latency)+cutoff), abs(mean_latency) + cutoff) 
    plots[0,0].set_xlabel("Latency (ms)")
    plots[0,0].set_ylabel("Count")
    plots[0,0].set_title(f"Fancy Latency Histogram")
    plots[0,0].legend()

    plots[0,1].hist(latencies, bins=12)
    plots[0,1].set_title("Latency Histogram")
    plots[0,1].set_xlabel("Latency (ms)")
    plots[0,1].set_ylabel("Count")

    plots[1,0].hist(isis[start_i], bins=50);
    plots[1,0].set_title("Sender ISI")
    plots[1,0].set_xlabel("Time bin(ms)")
    plots[1,0].set_ylabel("ISI count")

    plots[1,1].hist(isis[end_i], bins=50);
    plots[1,1].set_title("Receiver ISI")
    plots[1,1].set_xlabel("Time bin(ms)")
    plots[1,1].set_ylabel("ISI count")
    
    
    plots[2,0].plot( sd.neuron_data[0][start_i]["template"] )
    plots[2,0].set_title("Sender Spike Waveform")
    plots[2,0].set_xlabel("")
    plots[2,0].set_ylabel("")
    
    plots[2,1].plot( sd.neuron_data[0][end_i]["template"] )
    plots[2,1].set_title("Receiver Spike Waveform")
    plots[2,1].set_xlabel("")
    plots[2,1].set_ylabel("")
    

## <font color="blue">General Latency pair analysis

In [13]:
def plotPairAnalysis(start_i, end_i, sd):
    
    # Layout Plot
    import warnings
    warnings.filterwarnings("ignore")
    plotPairFootprint( start_i, end_i, sd )
    
    # Summary Stats
    latencies_raw = get_latencies( start_i, end_i, sd  )
    latencies_clean = latencies_raw[ np.where( np.abs(latencies_raw) < 15 )[0] ]
    print( "Number of Latencies", len(latencies_clean) )
    print( "Mean Latency", round(np.mean(latencies_clean), 3) )
    print( "Median Latency", round(np.median(latencies_clean), 3) )
    #print( "Latency Probability", {round(100*(len(lates_filtered)/len(lates_raw)))})
    print("STTC", sd.spike_time_tiling( start_i, end_i) )
    print("Diptest P-val", round( diptest.diptest(latencies_clean)[1] , 3) )
    print( "Latency Probability", round(len(latencies_clean)/len(latencies_raw), 3) )
    print(".")
    print( "Sender Neuron", start_i)
    print( "Sender   Firing Rate", round(sd.rates(unit='Hz')[start_i] , 3) )
    print(".")
    print("Receiver Neuron", end_i)
    print( "Receiver Firing Rate", round(sd.rates(unit='Hz')[end_i], 3) )
    
    # Plot other graphs
    figs, plots = plt.subplots(nrows=3,ncols=2,figsize=(14,10))

    mean_latency = np.mean(latencies_clean)     # Get mean and SD
    std_latency = np.std(latencies_clean)
    cutoff = 2 * std_latency           # remove outliers
    lates_filtered = [latency for latency in latencies_clean if abs(latency - mean_latency) <= cutoff]
    plots[0,0].hist(lates_filtered, bins=12, alpha=0.7, label='Latency')
    plots[0,0].axvline(mean_latency, color='red', linestyle='dashed', linewidth=2, label='Mean')
    plots[0,0].axvline(mean_latency - std_latency, color='green', linestyle='dashed', linewidth=2, label='Std -')
    plots[0,0].axvline(mean_latency + std_latency, color='green', linestyle='dashed', linewidth=2, label='Std +')
    plots[0,0].axvline(0, color='black', linestyle='dashed', linewidth=0.5, label='Std +')
    plots[0,0].set_xlim(-1*(abs(mean_latency)+cutoff), abs(mean_latency) + cutoff) 
    plots[0,0].set_xlabel("Latency (ms)")
    plots[0,0].set_ylabel("Count")
    plots[0,0].set_title(f"Fancy Latency Histogram")
    plots[0,0].legend()

    plots[0,1].hist(latencies_clean, bins=12)
    plots[0,1].set_title("Latency Histogram")
    plots[0,1].set_xlabel("Latency (ms)")
    plots[0,1].set_ylabel("Count")

    plots[1,0].hist(sd.interspike_intervals()[start_i], bins=50);
    plots[1,0].set_title("Sender ISI")
    plots[1,0].set_xlabel("Time bin(ms)")
    plots[1,0].set_ylabel("ISI count")

    plots[1,1].hist(sd.interspike_intervals()[end_i], bins=50);
    plots[1,1].set_title("Receiver ISI")
    plots[1,1].set_xlabel("Time bin(ms)")
    plots[1,1].set_ylabel("ISI count")
    
    
    plots[2,0].plot( sd.neuron_data[0][start_i]["template"] )
    plots[2,0].set_title("Sender Spike Waveform")
    plots[2,0].set_xlabel("")
    plots[2,0].set_ylabel("")
    
    plots[2,1].plot( sd.neuron_data[0][end_i]["template"] )
    plots[2,1].set_title("Receiver Spike Waveform")
    plots[2,1].set_xlabel("")
    plots[2,1].set_ylabel("")
    